In [ ]:
import os
import random
import shutil

# Chemin vers le dossier du dataset
dataset_path = 'test'  # Remplacez par le chemin de votre dataset
# Chemin vers le dossier de sortie
output_path = 'a_importer'  # Dossier de sortie

# Dossiers sources
images_source_dir = os.path.join(dataset_path, 'images')       # Dossier source pour les images
labels_source_dir = os.path.join(dataset_path, 'labels')   # Dossier source pour les annotations

# Dossiers de sortie pour les images et les annotations
images_output_dir = os.path.join(output_path, 'images')         # Dossier pour les images
labels_output_dir = os.path.join(output_path, 'labels')     # Dossier pour les annotations

# Créer les dossiers de sortie s'ils n'existent pas
os.makedirs(images_output_dir, exist_ok=True)
os.makedirs(labels_output_dir, exist_ok=True)

# Lister toutes les images dans le dossier source des images
all_images = [f for f in os.listdir(images_source_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]  # Ajouter d'autres formats si nécessaire

# Calculer le nombre d'images à sélectionner (10 %)
num_images_to_select = max(1, 50)  # Au moins 1 image si le dataset est très petit

# Sélectionner aléatoirement 10 % des images
selected_images = random.sample(all_images, num_images_to_select)

# Copier les images et leurs fichiers d'annotation correspondants dans les dossiers de sortie
for image in selected_images:
    # Copier l'image
    shutil.copy(os.path.join(images_source_dir, image), os.path.join(images_output_dir, image))

    # Copier le fichier d'annotation correspondant
    label_file = os.path.splitext(image)[0] + '.txt'  # Remplace l'extension de l'image par .txt
    if os.path.exists(os.path.join(labels_source_dir, label_file)):  # Vérifie si le fichier d'annotation existe
        shutil.copy(os.path.join(labels_source_dir, label_file), os.path.join(labels_output_dir, label_file))

print(f"{num_images_to_select} images et leurs annotations sélectionnées et copiées vers {images_output_dir} et {labels_output_dir}.")


50 images et leurs annotations sélectionnées et copiées vers C:/Users/YANISOUUUUUUUUGERARD/Desktop/opencv_train/to_import\images et C:/Users/YANISOUUUUUUUUGERARD/Desktop/opencv_train/to_import\labels.


: 

In [ ]:
import cv2
import os
from ultralytics import YOLO

# Charger le modèle
model = YOLO('lien_best.pt')
# Charger la vidéo
video_path = 'video_entré'
output_path = 'video_sortie'
save_dir = '../buts'  # Nouveau dossier pour les buts

# Créer le dossier s'il n'existe pas
os.makedirs(save_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)

# Paramètres pour l'enregistrement de la vidéo de sortie
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec vidéo
out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

def est_but(cage_coords, ballon_coords):
    cx_min, cy_min, cx_max, cy_max = cage_coords
    bx_min, by_min, bx_max, by_max = ballon_coords
    print(f"Vérification du but : Cage ({cx_min}, {cy_min}, {cx_max}, {cy_max}), Ballon ({bx_min}, {by_min}, {bx_max}, {by_max})")

    if bx_min > cx_min and bx_max < cx_max and by_min > cy_min and by_max < cy_max:
        return True
    return False

frame_count = 0  # Compteur de frames pour le nom des fichiers

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Appliquer le modèle YOLO sur chaque frame
    results = model.predict(source=frame)

    # Initialisation des coordonnées de la cage et du ballon
    cage_coords = None
    ballon_coords = None

    # Extraction des coordonnées de chaque détection
    for detection in results[0].boxes:
        classe = int(detection.cls)  # ID de classe sous forme d'entier
        class_name = model.names[classe]  # Obtenir le nom de la classe
        x_min, y_min, x_max, y_max = map(int, detection.xyxy[0])  # Coordonnées de la boîte

        print(f"Classe: {class_name}, Coordonnées: ({x_min}, {y_min}, {x_max}, {y_max})")

        if class_name == 'cage':
            cage_coords = (x_min, y_min, x_max, y_max)
        elif class_name == 'ball':  # Utiliser "ball" au lieu de "ballon"
            ballon_coords = (x_min, y_min, x_max, y_max)

    # Vérifier s'il y a but
    if cage_coords and ballon_coords:
        if est_but(cage_coords, ballon_coords):
            print("But détecté !")
            cv2.putText(frame, "But détecté !", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            # Sauvegarder l'image de but
            frame_filename = os.path.join(save_dir, f'but_detecte_{frame_count}.jpg')
            cv2.imwrite(frame_filename, frame)
            frame_count += 1

    # Dessiner les rectangles pour la cage et le ballon
    if cage_coords:
        cv2.rectangle(frame, (cage_coords[0], cage_coords[1]), (cage_coords[2], cage_coords[3]), (255, 0, 0), 2)
    if ballon_coords:
        cv2.rectangle(frame, (ballon_coords[0], ballon_coords[1]), (ballon_coords[2], ballon_coords[3]), (0, 0, 255), 2)

    # Afficher les résultats sur l'image
    annotated_frame = results[0].plot()

    # Écrire le frame annoté dans la vidéo de sortie
    out.write(annotated_frame)

    # Afficher la frame annotée
    cv2.imshow('Détection', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
out.release()
cv2.destroyAllWindows()